In [1]:
import transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import pandas as pd

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# model.summary()
# Now that we have our model, let us create our input sequences from the IMDB reviews dataset.
# IMDB reviews dataset is  a large movie review dataset collected and prepared by Andrew.

2023-06-12 17:13:32.375498: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 17:13:34.495724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 17:13:34.496336: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
All model checkpoint layers were used when initializing TFB

In [2]:
URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz", 
                                  origin=URL,
                                  untar=True,
                                  cache_dir='.',
                                  cache_subdir='')
# Removing the unlabeled reviews

import os
import shutil
import numpy as np

# Create main directory path ("/aclImdb")
main_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

# Create sub directory path ("/aclImdb/train")
train_dir = os.path.join(main_dir, 'train')

# Remove unsup folder since this is a supervised learning task
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

# View the final train folder
print(os.listdir(train_dir))



['urls_pos.txt', 'urls_unsup.txt', 'unsupBow.feat', 'neg', 'labeledBow.feat', 'pos', 'urls_neg.txt']


In [3]:
# Train and Test Split 
train = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train',\
    batch_size=30000,validation_split=0.2, subset='training', seed = 123)
test = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train',
    batch_size=30000, validation_split=0.2, subset='validation', seed = 123)

for i in train.take(1):
  train_feat = i[0].numpy()
  train_lab = i[1].numpy()

train = pd.DataFrame([train_feat, train_lab]).T
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
train['DATA_COLUMN'] = train['DATA_COLUMN'].str.decode("utf-8")
print(train.head())

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
                                         DATA_COLUMN LABEL_COLUMN
0  Canadian director Vincenzo Natali took the art...            1
1  I gave this film 10 not because it is a superb...            1
2  I admit to being somewhat jaded about the movi...            1
3  For a long time, 'The Menagerie' was my favori...            1
4  A truly frightening film. Feels as if it were ...            0
